# Introduction to Promises and PromiseTensors


### Context 

We introduce here Promises and, more specificly, PromiseTensors. The principle is quite similar to promises or futures that you may have encountered in some programming languages as Javascript. As stated in wikipedia, "they describe an object that acts as a proxy for a result that is initially unknown, usually because the computation of its value is not yet complete."

In particular, we will explore PromiseTensors which inherit from the Promise class and are used when the object that is waited for is a torch tensor. 

Let's write some code!

Author:
- Jason Paumier - GitHub: [@Jasopaum](https://github.com/https://github.com/Jasopaum)

### Imports and setup

First let's make the classic imports and hooking.

In [1]:
import torch
import syft as sy

hook = sy.TorchHook(torch)

Also one important note: **the local worker should not be a client worker.** *Non client workers can store objects and we need this ability to use promises.*

In [2]:
hook.local_worker.is_client_worker = False

### Creation and basic methods

Let's see how to create a PromiseTensor and use its basic methods.

In [3]:
# We create a promise for a FloatTensor
a = sy.Promise.FloatTensor(shape=torch.Size((2, 2)))

# Now, this promise can be kept as follows
ta = torch.tensor([[1., 2.], [3., 4.]])
a.keep(ta)

# Finally, we can retrieve the value
a.value()

tensor([[1., 2.],
        [3., 4.]])

In [4]:
# We can also keep a promise several times before getting the value
a.keep(torch.tensor([[5., 6.], [7., 8.]]))
a.keep(torch.tensor([[8., 7.], [6., 5.]]))

# The subsequent calls to .value() will give the tensors in the same order as they arrived
print(a.value())
print(a.value())

tensor([[5., 6.],
        [7., 8.]])
tensor([[8., 7.],
        [6., 5.]])


### Operations with promises

We can perform mainstream operations with promises.

In [5]:
a = sy.Promise.FloatTensor(shape=torch.Size((2, 2)))
b = sy.Promise.FloatTensor(shape=torch.Size((2, 2)))

promised_result = a + b
# The result of an operation involving promises is a promise itself:
print("promised_result: ", promised_result)

a.keep(torch.tensor([[1., 1.], [1., 1.]]))
a.keep(torch.tensor([[2., 2.], [2., 2.]]))
b.keep(torch.tensor([[3., 3.], [3., 3.]]))
b.keep(torch.tensor([[4., 4.], [4., 4.]]))

print(promised_result.value())
print(promised_result.value())

promised_result:  (Wrapper)>[PromiseTensor(me:31223276767) -future-> FloatTensor -blocking-> 0 plans]
tensor([[4., 4.],
        [4., 4.]])
tensor([[6., 6.],
        [6., 6.]])


Operations between promises and basic torch tensors are also possible.

In [6]:
a = sy.Promise.FloatTensor(shape=torch.Size((2, 2)))

promised_result = 2 * a

a.keep(torch.tensor([[1., 2.], [3., 4.]]))

print(promised_result.value())

tensor([[2., 4.],
        [6., 8.]])


### Example on a remote worker

PromiseTensors can also be used between different workers.

In [7]:
bob = sy.VirtualWorker(hook, id="bob")

In [8]:
a = sy.Promise.FloatTensor(shape=torch.Size((3, 3)))
b = sy.Promise.FloatTensor(shape=torch.Size((3, 3)))

x = a.send(bob)  # Here, x and y are pointers to PromiseTensors
y = b.send(bob)  # located on worker Bob

res = x - y  # res is a pointer to a PromiseTensor located on Bob
print(res)
print(bob._objects[res.id_at_location])

x.keep(torch.tensor([[1., 1.], [1., 1.]]))
y.keep(torch.tensor([[2., 3.], [4., 5.]]))

res.value().get()

(Wrapper)>[PointerTensor | me:40372024233 -> bob:33749923938]
(Wrapper)>[PromiseTensor(bob:33749923938) -future-> FloatTensor -blocking-> 0 plans]


tensor([[-1., -2.],
        [-3., -4.]])

Et voilà! We have seen how to use the powerful tool that are promises with PySyft. 

### Star PySyft on GitHub

The easiest way to help our community is just by starring the repositories! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Pick our tutorials on GitHub!

We made really nice tutorials to get a better understanding of what Federated and Privacy-Preserving Learning should look like and how we are building the bricks for this to happen.

- [Checkout the PySyft tutorials](https://github.com/OpenMined/PySyft/tree/master/examples/tutorials)


### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! 

- [Join slack.openmined.org](http://slack.openmined.org)

### Join a Code Project!

The best way to contribute to our community is to become a code contributor! If you want to start "one off" mini-projects, you can go to PySyft GitHub Issues page and search for issues marked `Good First Issue`.

- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

- [Donate through OpenMined's Open Collective Page](https://opencollective.com/openmined)